In [1]:

import numpy as np
import pandas as pd
from numpy import array
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os
import string
os.environ['KERAS_BACKEND']='tensorflow'

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model, Sequential

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from pickle import dump



Using TensorFlow backend.


In [2]:
def gpu_alloc(device_id):
    os.environ["CUDA_VISIBLE_DEVICES"]=device_id
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    set_session(tf.Session(config=config))

In [3]:
gpu_alloc("1")

# Data

In [ ]:
dat_path = '../dat/imdb/'
dat_file_name = 'imdb_master.csv'
dat_file = os.path.join(dat_path, dat_file_name)

In [ ]:
def load_data(dat_file):
    df = pd.read_csv(dat_file)
    train_data = df.train

In [32]:
from keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data()



From indices to txt

In [33]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [36]:
len(word_index)

88584

In [34]:

def int2txt(data, reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])):
    #reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
    #Decodes the review. Note that the indices are offset by 3 because 0, 1, and 2 are reserved indices for “padding,” “start of sequence,” and “unknown.”
    words = []
    for i in data:
        if i == 0:
            words.append('PAD')
        elif i == 1:
            words.append('START')
        elif i == 2 or i == 3:
            words.append('?')
        else:
            words.append(reverse_word_index[i])
    return ' '.join(words)#' '.join([reverse_word_index.get(i - 3, '?') for i in data])
    



                               

In [27]:
reverse_word_index[0]

KeyError: 0

In [35]:
for i in range(10):
    print(int2txt(train_data[i], reverse_word_index)) 

START as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room titillate it so heart shows to years of every never going villaronga help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but pratfalls to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other tricky in of seen over landed for anyone of gilmore's br show's to whether from than out themselves history he name half some br of 'n odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but w

# NLM
https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/

## Data preprocessing

In [ ]:
def prepare_lm_data(texts, seq_len):
    sequences = tokenizer.texts_to_sequences(texts) 
    # organize into sequences of tokens
    length = seq_len + 1
    sequences = list()
    for i in range(length, len(tokens)):
        # select sequence of tokens
        seq = tokens[i-length:i]
        # convert into a line
        line = ' '.join(seq)
        # store
        sequences.append(line)    

In [18]:
# load
in_filename = out_filename
doc = load_doc(in_filename)
lines = doc.split('\n')

# integer encode sequences of words
#tokenizer = Tokenizer(num_words=10000)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
word_index = tokenizer.word_index
print(vocab_size)



9741


In [19]:
sz_limit = len(sequences)
# separate into input and output
sequences = array(sequences[:sz_limit])
X, y = sequences[:,:-1], sequences[:,-1]
#y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [ ]:
# define model
#model = Sequential()
#model.add(Embedding(vocab_size, 50, input_length=seq_length))
#model.add(LSTM(100, return_sequences=True))
#model.add(LSTM(100))

GLOVE_DIR = "./dat/glove"
EMBEDDING_DIM = 100
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()

print('Total %s word vectors.' % len(embeddings_index))

embedding_matrix = np.random.random((vocab_size, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(vocab_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=True)
sentence_input = Input(shape=(seq_length,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_word_enc = TimeDistributed(Dense(200))(l_lstm)

#model.add(Dense(100, activation='relu'))
l_dense = Dense(100, activation='relu')(l_word_enc)
#model.add(Dense(vocab_size, activation='softmax'))
output = Dense(vocab_size, activation='softmax')(l_dense)
model = Model(sentence_input, output)
print(model.summary())
word_enc = Model(sentence_input, l_word_enc)

# compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 50)            487050    
_________________________________________________________________
lstm_5 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 9741)              983841    
Total params: 1,621,791
Trainable params: 1,621,791
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# fit model
model.fit(X, y, batch_size=128, epochs=100)

# save the model to file
lm_model_file_name = 'imdb_keras_lm_model.h5'
model.save(lm_model_file_name)
# save the tokenizer
tok_file_name = 'imdb_keras_tokenizer.pkl'
dump(tokenizer, open(tok_file_name, 'wb'))
word_enc_model_file_name = 'word_enc_model.h5'
word_enc.save(word_enc_model_file_name)

Epoch 1/100
  383872/10976162 [>.............................] - ETA: 5:14:00 - loss: 6.3967 - acc: 0.0818

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 1322624/10976162 [==>...........................] - ETA: 4:45:13 - loss: 6.0646 - acc: 0.1098

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



10976162/10976162 [==============================] - 16708s 2ms/step - loss: 5.5343 - acc: 0.1476
Epoch 2/100
 7204736/10976162 [==================>...........] - ETA: 1:27:22 - loss: 5.2872 - acc: 0.1667

...

  649216/10976162 [>.............................] - ETA: 2:38:58 - loss: 5.1623 - acc: 0.1770

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 1346432/10976162 [==>...........................] - ETA: 2:28:13 - loss: 5.1619 - acc: 0.1769

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



10976162/10976162 [==============================] - 10079s 918us/step - loss: 5.1224 - acc: 0.1809
Epoch 7/100
10976162/10976162 [==============================] - 10116s 922us/step - loss: 5.1073 - acc: 0.1821
Epoch 8/100
 2490496/10976162 [=====>........................] - ETA: 2:11:04 - loss: 5.0931 - acc: 0.1831

## Use LM

In [ ]:
# Use LM
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences


# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

# load cleaned text sequences

in_filename = out_filename
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model(lm_model_file_name)

# load the tokenizer
tokenizer = load(open(tok_file_name, 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)